In [3]:
import pandas as pd

C:\Users\Selecty\AppData\Local\Temp\ipykernel_17880\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### **Задание 1.**
Рассчитайте прирост средней коверсии на 1 магазин в Санкт-Петербурге из входящих в покупатели. Данные находятся в файле "task_1"
В анализе прироста средней конверсии Периода 2 к Периоду 1 должны участвовать только сопоставимые магазины (LFL) - работавшие в обоих периодах.

In [30]:
task_1_dataset = pd.read_csv('task_1.csv', sep = ';')

In [31]:
# Выделим два периода
period_1 = task_1_dataset[task_1_dataset['Период']==1]
period_2 = task_1_dataset[task_1_dataset['Период']==2]

# объеденим два периода в общий датасет
conv_dataset = period_1.merge(period_2, on=['Код магазина'], how='inner') 
# Отфильтруем странные значения, где 0 входящих
conv_dataset = conv_dataset[(conv_dataset['Входящие, чел_x'] != 0) & (conv_dataset['Входящие, чел_y'] != 0)]


In [41]:
# Посчитаем прирост конверсий
conv_dataset['conv_1'] = conv_dataset['Покупатели, чел._x'] / conv_dataset['Входящие, чел_x']
conv_dataset['conv_2'] = conv_dataset['Покупатели, чел._y'] / conv_dataset['Входящие, чел_y']
conv_dataset['conv_diff'] = (conv_dataset['conv_2'] - conv_dataset['conv_1'])

**Готово**

#### **Задание 2.**
 	
Рассчитайте наценку на ювелирные изделия по данным в файле "task_2". Соблюсти следующие условия:
1. Период анализа - с 1.04.23 по 10.04.23
2. В анализе должны участвовать только продажи изделий с общей суммой и себестоимостью больше 0 рублей
3. В ответе необходимо получить два значения наценки - среднюю наценку на ювелирные изделия из золота и среднюю наценку на ювелирные изделия из серебра. Сгруппировать изделия по металлу нужно на основе поля "Товарная группа". Товарная группа изделий из серебра начинается на "СИ", все остальные изделия - это золото.

In [107]:
task_2_dataset = pd.read_csv('task_2.csv', sep = ';')

In [108]:
# Подправим период анализа
task_2_dataset['ДатаПродажи'] = pd.to_datetime(task_2_dataset['ДатаПродажи'], format='%d.%m.%Y %H:%M')

# Задаем даты начала и конца интервала
start_date = pd.to_datetime('2023-04-01')
end_date = pd.to_datetime('2023-04-16')
task_2_dataset = task_2_dataset[(task_2_dataset['ДатаПродажи'] >= start_date) & (task_2_dataset['ДатаПродажи'] <= end_date)]

#Отфильтруем продажи изделий с общей суммой и себестоимостью больше 0 рублей
task_2_dataset = task_2_dataset[(task_2_dataset['ОбщаяСумма'] > 0) & (task_2_dataset['Себестоимость'] > 0)]

#Расчитаем наценку
task_2_dataset['markup'] = task_2_dataset['ОбщаяСумма'] - task_2_dataset['Себестоимость']

In [109]:
# Функция для определения категории изделия - золото и серебро
def categorize_product(group):
    if group.startswith('СИ'):
        return 'серебро'
    else:
        return 'золото'

task_2_dataset['Категория'] = task_2_dataset['ТоварнаяГруппа'].apply(categorize_product)

In [110]:
#Подсчитаем среднюю по группам
grouped_mean = task_2_dataset.groupby('Категория')['markup'].mean()
print(grouped_mean)

Категория
золото     9066.940511
серебро    1330.022642
Name: markup, dtype: float64
